In [32]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, roc_curve, auc
from sklearn.model_selection import train_test_split, TimeSeriesSplit
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.preprocessing import FunctionTransformer, StandardScaler, MinMaxScaler
from sklearn.svm import LinearSVC, SVC
from sklearn.decomposition import TruncatedSVD
from sklearn.base import TransformerMixin, BaseEstimator
from sklearn.neighbors import KNeighborsClassifier
from sklearn.utils import shuffle
from text_clean import *


import numpy as np
import pylab as pl
import matplotlib.pyplot as plt
import pandas as pd


def label_stocks(row):
    if row['Close'] >= row['yesterday_close']:
        return 1
    else:
      return 0


apple_stocks = "Data/Stock data/Stocks/aapl.us.txt"
amazon_stocks = "Data/Stock data/Stocks/amzn.us.txt"
facebook_stocks = "Data/Stock data/Stocks/fb.us.txt"

apple_headlines = "Data/News data/apple_deduped.csv"
amazon_headlines = "Data/News data/amazon_deduped.csv"
facebook_headlines = "Data/News data/facebook_deduped.csv"

apple_reddit = "Data/News data/apple_reddit.csv"
amazon_reddit = "Data/News data/amazon_reddit.csv"
facebook_reddit = "Data/News data/facebook_reddit.csv"

apple_tweets = "Data/Tweets/apple_tweets.csv"
amazon_tweets = "Data/Tweets/amazon_tweets.csv"
facebook_tweets = "Data/Tweets/facebook_tweets.csv"

mydateparser = lambda x: pd.datetime.strptime(x[:10] + " " + x[-4:], "%a %b %d %Y")
# df = pd.read_csv('Data/Tweets/apple_tweets.csv', parse_dates=[1], date_parser=mydateparser)
df = pd.read_csv(apple_headlines, parse_dates=[1])
stock = pd.read_csv('aapl_indicators.us.txt', parse_dates=[0])
stock['yesterday_close'] = stock['Close'].shift()
stock['value'] = stock.apply(lambda row: label_stocks(row), axis=1)
stock.loc[:, 'High':'others_cr'] = stock.loc[:, 'High':'others_cr'].shift()

In [33]:
# stock['value'] = stock.apply(lambda row: label_stocks2(row), axis=1)
# stock.loc[:, 'High': 'others_cr'] = stock.loc[:, 'High': 'others_cr'].shift()
# label_stocks1(df)

# df.rename({'created_at': 'Date'}, inplace=True, axis=1)
df = pd.merge(df, stock, on="Date", how="inner")
df.drop('Close', axis=1, inplace=True)
# df.drop('Date', axis=1, inplace=True)
# df.drop('user_id_str', axis=1, inplace=True)
df = df.set_index(['Date'])

In [34]:
scaler = MinMaxScaler()
df.loc[:, 'Open': 'yesterday_close'] = scaler.fit_transform(df.loc[:, 'Open': 'yesterday_close'])

In [40]:
stock

,Date,Open,High,Low,Close,Volume,volume_adi,volume_obv,volume_obvm,volume_cmf,...,momentum_uo,momentum_stoch,momentum_stoch_signal,momentum_wr,momentum_ao,others_dr,others_dlr,others_cr,yesterday_close,value
0,1984-09-07,0.42388,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
1,1984-09-10,0.42388,0.42902,0.41874,0.42388,23220030.0,0.000000e+00,0.0,0.0,0.000000,...,50.000000,50.000000,50.000000,-50.000000,0.000000,0.000000,0.000000,0.000000,0.42388,0
2,1984-09-11,0.42516,0.42516,0.41366,0.42134,18022532.0,6.049302e+06,-18022532.0,0.0,0.000000,...,50.000000,50.000000,50.000000,-50.000000,0.000000,-0.599226,-0.601029,-0.599226,0.42134,1
3,1984-09-12,0.42902,0.43668,0.42516,0.42902,42498199.0,-7.969201e+06,42498199.0,0.0,0.000000,...,50.000000,50.000000,50.000000,-50.000000,0.000000,1.822756,1.806343,1.212607,0.42902,0
4,1984-09-13,0.43927,0.43157,0.41618,0.41618,37125801.0,-5.114430e+07,-37125801.0,0.0,0.000000,...,50.000000,50.000000,50.000000,-50.000000,0.000000,-2.992867,-3.038568,-1.816552,0.41618,1
5,1984-09-14,0.44052,0.44052,0.43927,0.43927,57822062.0,-9.494786e+07,57822062.0,0.0,0.000000,...,50.000000,50.000000,50.000000,-50.000000,0.000000,5.548080,5.399640,3.630745,0.43927,1
6,1984-09-17,0.45718,0.45589,0.44052,0.44566,68847968.0,-8.062207e+07,68847968.0,0.0,0.000000,...,50.000000,50.000000,50.000000,-50.000000,0.000000,1.454686,1.444207,5.138247,0.44566,1
7,1984-09-18,0.45718,0.46357,0.45718,0.45718,53755262.0,-7.655527e+07,53755262.0,0.0,0.000000,...,50.000000,50.000000,50.000000,-50.000000,0.000000,2.584930,2.552086,7.855997,0.45718,0
8,1984-09-19,0.44052,0.46103,0.44052,0.44052,27136886.0,-8.089215e+07,-27136886.0,0.0,0.000000,...,50.000000,50.000000,50.000000,-50.000000,0.000000,-3.644079,-3.712134,3.925639,0.44052,0
9,1984-09-20,0.43286,0.44566,0.43157,0.43157,29641922.0,-5.677881e+07,-29641922.0,0.0,0.000000,...,50.000000,50.000000,50.000000,-50.000000,0.000000,-2.031690,-2.052613,1.814193,0.43157,1


In [42]:

get_text_data = FunctionTransformer(lambda x: x['text'], validate=False)
get_numeric_data = FunctionTransformer(lambda x: x.loc[:, 'Open': 'yesterday_close'], validate=False)

vect = TfidfVectorizer(analyzer=preprocess_text, strip_accents='unicode', ngram_range=(1, 3))

X = df.loc[:, 'text': 'yesterday_close']
y = df.loc[:, 'value']

# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train, y_train = shuffle(X.loc[:'2015-07-26'], y.loc[:'2015-07-26'])
X_test, y_test = shuffle(X.loc['2015-07-27':], y.loc['2015-07-27':])

model = Pipeline([
    ('features', FeatureUnion([
            ('numeric_features', Pipeline([
                ('selector', get_numeric_data)
            ])),
               ('text_features', Pipeline([
                  ('selector', get_text_data),
                  ('vec',  vect)
              ]))
         ])),
    #('clf', LinearSVC(class_weight='balanced', max_iter=2000, penalty='l1', dual=False)),
    ('clf', MultinomialNB()),
    #('clf', LinearSVC(class_weight='balanced', max_iter=2000, penalty='l1', dual=False)),
])

model.fit(X_train, y_train)
print(model.score(X_train, y_train))
y_pred = model.predict(X_test)

print("Accuracy: {:.2f}%".format(accuracy_score(y_test, y_pred) * 100))
print("\nF1 Score: {:.2f}".format(f1_score(y_test, y_pred) * 100))

print("\nConfusion Matrix:\n", confusion_matrix(y_test, y_pred), "\n")

0.7755672476159158
Accuracy: 53.17%

F1 Score: 44.60

Confusion Matrix:
 [[1038  620]
 [ 796  570]] 



# Graph most influential words:

In [27]:
def top_tfidf_feats(row, features, top_n=25):
    ''' Get top n tfidf values in row and return them with their corresponding feature names.'''
    topn_ids = np.argsort(row)[::-1][:top_n]
    top_feats = [(features[i], row[i]) for i in topn_ids]
    df = pd.DataFrame(top_feats)
    df.columns = ['feature', 'tfidf']
    return df

def plot_tfidf_classfeats_h(dfs):
    ''' Plot the data frames returned by the function plot_tfidf_classfeats(). '''
    fig = plt.figure(figsize=(12, 9), facecolor="w")
    x = np.arange(len(dfs[0]))
    for i, df in enumerate(dfs):
        ax = fig.add_subplot(1, len(dfs), i+1)
        ax.spines["top"].set_visible(False)
        ax.spines["right"].set_visible(False)
        ax.set_frame_on(False)
        ax.get_xaxis().tick_bottom()
        ax.get_yaxis().tick_left()
        ax.set_xlabel("Mean Tf-Idf Score", labelpad=16, fontsize=14)
        ax.set_title("label = " + str(df.label), fontsize=16)
        ax.ticklabel_format(axis='x', style='sci', scilimits=(-2,2))
        ax.barh(x, df.tfidf, align='center', color='#3F5D7D')
        ax.set_yticks(x)
        ax.set_ylim([-1, x[-1]+1])
        yticks = ax.set_yticklabels(df.feature)
        plt.subplots_adjust(bottom=0.09, right=0.97, left=0.15, top=0.95, wspace=0.52)
    plt.show()
    
def top_mean_feats(Xtr, features, grp_ids=None, min_tfidf=0.1, top_n=25):
    ''' Return the top n features that on average are most important amongst documents in rows
        indentified by indices in grp_ids. '''
    if grp_ids:
        D = Xtr[grp_ids].toarray()
    else:
        D = Xtr.toarray()

    D[D < min_tfidf] = 0
    tfidf_means = np.mean(D, axis=0)
    return top_tfidf_feats(tfidf_means, features, top_n)


def top_feats_by_class(Xtr, y, features, min_tfidf=0.1, top_n=25):
    ''' Return a list of dfs, where each df holds top_n features and their mean tfidf value
        calculated across documents with the same class label. '''
    dfs = []
    labels = np.unique(y)
    for label in labels:
        ids = np.where(y==label)
        feats_df = top_mean_feats(Xtr, features, ids, min_tfidf=min_tfidf, top_n=top_n)
        feats_df.label = label
        dfs.append(feats_df)
    return dfs

# model.named_steps['features'].transformer_list[1][1].named_steps['vec']
Xtr = model.named_steps['features'].transformer_list[1][1].named_steps['vec'].fit_transform(X['text'])
features = model.named_steps['features'].transformer_list[1][1].named_steps['vec'].get_feature_names()
feats = top_feats_by_class(Xtr, y, features)

plot_tfidf_classfeats_h(feats)

In [1]:
# Time Series split?
X = df.loc[:, 'text': 'yesterday_close']
y = df['value']
tscv = TimeSeriesSplit(n_splits=5)
for train_index, test_index in tscv.split(X):
    print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    # print(X_train)
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    # print(X_test)

NameError: name 'df' is not defined

In [ ]:
# Figure out how to graph SVM
svd = TruncatedSVD(n_components=2, random_state=42)
transformed = svd.fit_transform(model.named_steps["debug"].data)
principalDf = pd.DataFrame(data = transformed
             , columns = ['principal component 1', 'principal component 2'])
finalDf = pd.concat([principalDf, df[['value']]], axis = 1)